In [94]:
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import math 
from matplotlib import pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

In [24]:
import pickle
import tensorflow as tf

from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [4]:
def result(predictions):
    predictions = predictions.tolist()
    result_list=[]
    for pred in predictions:
        result = pred // 0.5 * 0.5
        if (pred - result) > 0.25:
            result += 0.5
        if result < 1.0:
            result = 1.0
        if result > 5.0:
            result = 5.0
        result_list.append(result)
    return result_list

# Accuracy score
def accuracy(Ypred, Ytrue):
    Ytrue = Ytrue.tolist()
    accurate = 0
    for i in range(len(Ytrue)):
        if Ytrue[i] == Ypred[i]:
            accurate += 1
    return accurate / len(Ytrue)

# approximate accurancy rate
def score(pred, test):
    test = test.tolist()
    correct = 0
    for i in range(len(test)):
        p = pred[i]
        t = test[i]
        if p < t+0.5 and p > t-0.5:
            correct += 1
    return correct / len(test)

# Ensemble Learning

linear reg = lr1, lr2
decision tree = dt1, dt2
neural network = nn1, nn2

In [96]:
lr1 = pickle.load(open("lr.sav", 'rb'))
lr2 = pickle.load(open("lr_second.sav", 'rb'))
dt1 = pickle.load(open("Decision_Tree_vocab.sav", 'rb'))
dt2 = pickle.load(open("Decision_Tree_cohesion.sav", 'rb'))
knn1 = pickle.load(open("knn_vocab.sav", 'rb'))
knn1 = pickle.load(open("knn_cohesion.sav", 'rb'))

#nn1 = pickle.load(open("./neural_network_vocab.sav", 'rb'))
#nn2 = pickle.load(open("./neural_network.sav", 'rb'))
#nn1 = tf.keras.models.load_model("./model_deep.h5")
#nn2 = tf.keras.models.load_model("./model_deep_cohesion.h5")

C:\Users\12560\anaconda3\envs\BT4222\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\12560\anaconda3\envs\BT4222\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\12560\anaconda3\envs\BT4222\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator KNeighborsRegressor from version 1.0.2 when using version 1.1.2. This might lead to br

## 1 Predict Vocabulary

In [51]:
df = pd.read_csv("Processed_Data.csv")
y = df['vocabulary']
X = df.iloc[:, 7:]
X = X.drop('corrected_text', axis = 1)

### 1.1 Voting

linear regression:
exact accuracy = 0.3987730061349693
approximate accuracy = 

decision tree:
exact accuracy = 0.4061302681992337
approximate accuracy =  0.7049808429118773

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)
vr = VotingRegressor(
    estimators=[('linear regressor', lr1), ('decision tree', dt1), ('KNN', knn1)]
)

In [102]:
vr.fit(X_train, y_train)
y_pred_vr = vr.predict(X_test)
print(accuracy(result(y_pred_vr), y_test))
print(score(y_pred_vr, y_test))

0.401840490797546
0.7208588957055214


### 1.2 AdaBoost

In [87]:
ar1 = AdaBoostRegressor(
    base_estimator=DecisionTreeRegressor(criterion='friedman_mse', max_depth=3, min_samples_split=2)
)
ar1.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='friedman_mse',
                                                       max_depth=3))

In [88]:
y_pred_ar1 = ar_dt1.predict(X_test)
print(accuracy(result(y_pred_ar1), y_test))
print(score(y_pred_ar1, y_test))

0.40899795501022496
0.7055214723926381


In [89]:
ar1.estimators_

[DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                       random_state=123178633),
 DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                       random_state=1801146387),
 DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                       random_state=456908786),
 DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                       random_state=1812362098),
 DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                       random_state=1082032842),
 DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                       random_state=1999456007),
 DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                       random_state=1231684026),
 DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                       random_state=843678351),
 DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                       random_state=374527278),
 Deci

In [90]:
ar1.estimator_weights_

array([1.37902863, 1.05462189, 1.15835248, 0.9711379 , 0.8174012 ,
       0.88050356, 0.6839971 , 0.82255339, 0.67311048, 0.93497022,
       0.67821407, 0.55925308, 0.28179957, 0.37779958, 0.82251179,
       0.51775457, 0.39731707, 0.36321335, 0.70790777, 0.39329742,
       0.54426494, 0.49757975, 0.60359166, 0.71322366, 0.86545968,
       0.74679942, 1.14369405, 0.72118245, 0.2754163 , 0.72904205,
       0.39048201, 0.63358064, 0.27781142, 0.44182905, 0.54740207,
       0.25180591, 0.68865011, 0.94706998, 0.63125329, 0.72338256,
       0.22571214, 0.6078989 , 0.58178252, 0.86583419, 0.463752  ,
       0.38458125, 0.45942464, 0.67518314, 0.36694063, 0.63935018])

# 1.3 Gradient Boost

Use the default decision tree in GradientBoostingRegressor.

Find best parameters for GradientBoostingRegressor.

In [95]:
gbr1 = GradientBoostingRegressor(n_estimators=300)
parameters = {
    'loss':['squared_error', 'absolute_error', 'huber', 'quantile'],
    'learning_rate':[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
    'criterion':['friedman_mse', 'squared_error', 'mse']
}
gs_gbr1 = GridSearchCV(gbr1, parameters)
print(gs_gbr1.best_params_)
print(gs_gbr1.best_score_)

C:\Users\12560\anaconda3\envs\BT4222\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\12560\anaconda3\envs\BT4222\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\12560\anaconda3\envs\BT4222\lib\site-packages\sklearn\ensemble\_gb.py", line 577, in fit
    self._check_params()
  File "C:\Users\12560\anaconda3\envs\BT4222\lib\site-packages\sklearn\ensemble\_gb.py", line 270, in _check_params
    check_scalar(
  File "C:\Users\125

{'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'huber'}
0.369309112331468


In [100]:
gbr1_adj = GradientBoostingRegressor(
    criterion = 'friedman_mse',
    learning_rate = 0.1,
    loss = 'huber',
    n_estimators=300
)
gbr1_adj.fit(X_train, y_train)
y_pred_gbr1_adj = gbr1_adj.predict(X_test)

print(accuracy(result(y_pred_gbr1_adj), y_test))
print(score(y_pred_gbr1_adj, y_test))

0.39775051124744376
0.7085889570552147


In [106]:
gbr1_1 = GradientBoostingRegressor(n_estimators=300)
gbr1_1.fit(X_train, y_train)
y_pred_gbr1_1 = gbr1_1.predict(X_test)
print(accuracy(result(y_pred_gbr1_1), y_test))
print(score(y_pred_gbr1_1, y_test))

0.4059304703476483
0.7198364008179959


Compare the default regressor and the fine tuned one. The default regressor works better.

Look at the feature importance.

In [110]:
fi_gbr1_1 = pd.DataFrame([gbr1_1.feature_names_in_, gbr1_1.feature_importances_]).T
fi_gbr1_1 = fi_gbr1_1.sort_values(by=1,ascending=False)
fi_gbr1_1.head(10)

,0,1
17,Incorrect_form_ratio,0.365601
23,number_of_diff_words,0.183728
26,coherence_score,0.038382
24,freq_diff_words,0.034787
31,freq_of_pronoun,0.022882
25,ttr,0.019596
27,lexrank_avg_min_diff,0.018763
0,number_of_words,0.01869
11,freq_of_wrong_words,0.018025
20,dale_chall_readability_score,0.017734


In [115]:
X_train_gbr1_1_res = X_train[['Incorrect_form_ratio','number_of_diff_words','coherence_score']]
y_train_gbr1_1_res = y_train
X_test_gbr1_1_res = X_test[['Incorrect_form_ratio','number_of_diff_words','coherence_score']]
y_test_gbr1_1_res = y_test
gbr1_1_res = GradientBoostingRegressor()
gbr1_1_res.fit(X_train_gbr1_1_res, y_train_gbr1_1_res)
y_pred_gbr1_1_res = gbr1_res.predict(X_test_gbr1_1_res)
print(accuracy(result(y_pred_gbr1_1_res), y_test_gbr1_1_res))
print(score(y_pred_gbr1_1_res, y_test_gbr1_1_res))

ValueError: X has 3 features, but GradientBoostingRegressor is expecting 2 features as input.

After combining 300 decision tree in 

### 1.3 Stacking

random forest

## 2 Predict Cohesion with Prediction of Vocabulary

### 2.1 Voting

linear regression:
exact accuracy = 0.33844580777096117
approximate accuracy = 

decision tree:
exact accuracy = 0.3282247765006386
approximate accuracy = 0.5900383141762452

In [20]:
y = df['cohesion']
X = df.iloc[:, 7:]
X = X.drop('corrected_text', axis = 1)
X['vocabulary'] = vr.predict(X)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
vr2 = VotingRegressor(
    estimators=[('linear regressor', lr2), ('decision tree', dt2)]
)

In [22]:
vr2.fit(X_train, y_train)
y_pred2 = vr2.predict(X_test)
print(accuracy(result(y_pred2), y_test))
print(score(y_pred2, y_test))

0.3343558282208589
0.6390593047034765


### 2.2 AdaBoost

### 2.3 Stacking